# Descarga del conjunto de datos

Código para automatizar la descarga de los registros eeg de la base de datos. Se encarga de descargar un archivo desde la URL especificada 

**Bibliotecas necesarias para realizar solicitudes HTTP y procesar HTML**

In [ ]:
import requests
from bs4 import BeautifulSoup

Realizamos solicitud GET con stream=True, lo que permite la descarga de grandes archivos en "trozos" sin necesidad de cargar todo el archivo en la memoria de una vez.

In [ ]:
def download_file(url, index):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()
    return local_filename

Descargamos desde el 1 al registro 109, con 14 registros por paciente. 

In [ ]:

for i in range(1, 110):

    if i < 10:
        root_link="https://archive.physionet.org/pn4/eegmmidb/S00" + str(i) + "/"
    elif i >= 10 and i < 100 :
        root_link = "https://archive.physionet.org/pn4/eegmmidb/S0" + str(i) + "/"
    else:
        root_link = "https://archive.physionet.org/pn4/eegmmidb/S" + str(i) + "/"

    r=requests.get(root_link)

    if r.status_code==200:
        soup=BeautifulSoup(r.text, features="html.parser")
        # print(soup.prettify())
        index=1
        for link in soup.find_all('a'):
            new_link=root_link+link.get('href')

            if new_link.endswith(".edf"):
                file_path=download_file(new_link,str(index))
                print("downloading:"+new_link+" -> "+file_path)
                index+=1

        print("all download finished")
      
    else:
        print("errors occur.")